In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [2]:
# read in data
train = pd.read_csv("facility_scores_known.csv")
test = pd.read_csv("facility_scores_unknown.csv")

# split training data into feature vectors and targets
train_y = train.loc[:, "inspection_score"]
train_x = train.drop("inspection_score", axis = 1)
test_y = test.loc[:, "inspection_score"]
test_x = test.drop("inspection_score", axis = 1)

train_x.shape, test_x.shape

((43199, 22), (10774, 22))

In [3]:
train_x

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_type,violation_id,violation_description,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
0,69618,Fancy Wheatfield Bakery,1362 Stockton St,San Francisco,CA,94133,NaN,NaN,NaN,NaN,...,Complaint,69618_20190304_103130,Inadequate sewage or wastewater disposal,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,...,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
2,69487,Hakkasan San Francisco,1 Kearny St,San Francisco,CA,94108,NaN,NaN,NaN,NaN,...,Routine - Unscheduled,69487_20180418_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
3,91044,Chopsticks Restaurant,4615 Mission St,San Francisco,CA,94112,NaN,NaN,NaN,NaN,...,Non-inspection site visit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,85987,Tselogs,552 Jones St,San Francisco,CA,94102,NaN,NaN,NaN,NaN,...,Routine - Unscheduled,85987_20180412_103132,Improper thawing methods,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43194,89569,Blue Bottle Coffee,2 South Park,San Francisco,CA,94107,NaN,NaN,NaN,NaN,...,Routine - Unscheduled,89569_20190506_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
43195,99764,POKE KANA,65 CAMBON Dr,San Francisco,CA,94132,NaN,NaN,NaN,NaN,...,New Ownership - Followup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43196,84541,Philz Coffee,300 Folsom St,San Francisco,CA,94105,NaN,NaN,NaN,NaN,...,Routine - Unscheduled,84541_20190506_103133,Foods not protected from contamination,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
43197,91572,El Gran Taco Loco,4591 Mission St.,San Francisco,CA,94112,NaN,NaN,NaN,NaN,...,Routine - Unscheduled,91572_20190506_103116,Inadequate food safety knowledge or lack of ce...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# fill in the null values in inspection_score with -1
train_y = train_y.fillna(-1)
train_y

0        -1.0
1        96.0
2        88.0
3        -1.0
4        94.0
         ... 
43194    80.0
43195    -1.0
43196    92.0
43197    76.0
43198    80.0
Name: inspection_score, Length: 43199, dtype: float64

In [5]:
# see which columns have nulls - (43199 row)
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43199 entries, 0 to 43198
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   business_id                43199 non-null  int64  
 1   business_name              43199 non-null  object 
 2   business_address           43199 non-null  object 
 3   business_city              43199 non-null  object 
 4   business_state             43199 non-null  object 
 5   business_postal_code       42245 non-null  object 
 6   business_latitude          22015 non-null  float64
 7   business_longitude         22015 non-null  float64
 8   business_location          22015 non-null  object 
 9   business_phone_number      14144 non-null  float64
 10  inspection_id              43199 non-null  object 
 11  inspection_date            43199 non-null  object 
 12  inspection_type            43199 non-null  object 
 13  violation_id               32218 non-null  obj

In [6]:
#### remove columns that have 45% of their data as null 

# get columns with 45% null
columns = train_x.columns
remove_col = []
for i in columns:
    if (train_x.loc[train_x[i].isna()].shape[0]) >= (train_x.shape[0]*0.45):
        remove_col.append(i)
        
# drop columns with 45% null       
train_x = train_x.drop(remove_col, axis = 1)
test_x = test_x.drop(remove_col, axis = 1)

train_x.shape, test_x.shape

((43199, 12), (10774, 12))

In [7]:
# check (relevant) unique values for columns with 43199 non-null values and remove columns with 100% similar values 

# train_x.business_id.nunique() # -> 6154          # restaurants appear multiple times
# train_x.business_name.nunique() # -> 5686        # probably not equal with business_id due to chain restaurants
# train_x.business_address.nunique() # -> 5602
# train_x.business_city.nunique() # -> 1           # remove column
# train_x.business_state.nunique() # -> 1          # remove column
# train_x.inspection_id.nunique() # -> 23252
# train_x.inspection_date.nunique() # -> 809
# train_x.inspection_type.nunique() # -> 14        # label encode

# remove business_city and business_state columns
train_x = train_x.drop(["business_city","business_state"], axis = 1)
test_x = test_x.drop(["business_city","business_state"], axis = 1)

train_x.shape, test_x.shape

((43199, 10), (10774, 10))

In [8]:
# deal with missing values: business_postal_code
train_x.business_postal_code.unique() 

array(['94133', '94118', '94108', '94112', '94102', '94114', '94103',
       '94107', '94110', '94104', nan, '94105', '94115', '94123', '94134',
       '94122', '94117', '94121', '94130', '94124', '94116', '94158',
       '94111', '94109', '94127', '94131', '95133', '94132', '94188',
       '95122', '95109', 'Ca', '94101', '95117', '94901', '941102019',
       '94602', '94013', '94080', '95132', '94143', '94544', '94122-1909',
       '94102-5917', '64110', '94301', '94105-1420', '94518',
       '94117-3504', '94123-3106', '94621', '94124-1917', '00000',
       '94129', '95105', '941033148', 'CA', '94105-2907', '94120',
       '95112', '94014', '941', '92672'], dtype=object)

In [9]:
#issues with business_postal_code:

########## we have "Ca", "CA", 941, and nan in business_postal_code

# all Ca values are for Tacolicious on Chestnut, zipcode = 94123
train_x.loc[train_x.business_postal_code == "Ca", "business_postal_code"] = "94123"
test_x.loc[test_x.business_postal_code == "Ca", "business_postal_code"] = "94123" 

# CA values for Japacurry, zipcode = 94103
train_x.loc[(train_x.business_postal_code == "CA") 
            & (train_x.business_name == "Japacurry"), "business_postal_code"] = "94103"

test_x.loc[(test_x.business_postal_code == "CA") 
           & (test_x.business_name == "Japacurry"), "business_postal_code"] = "94103"

# CA values for and Leo's Hot Dogs, zipcode =94110
train_x.loc[(train_x.business_postal_code == "CA")
            & (train_x.business_name == "Leo's Hot Dogs"), "business_postal_code"] = "94110" 

test_x.loc[(test_x.business_postal_code == "CA") 
           & (test_x.business_name == "Leo's Hot Dogs"), "business_postal_code"] = "94110"

# 941 is an error for GOLDEN GATE YACHT CLUB and zip code is 94123. This was not an error in test_x
train_x.loc[(train_x.business_postal_code == "941")
           & (train_x.business_name == "GOLDEN GATE YACHT CLUB"), "business_postal_code"] = "94123"

# train_x.loc[train_x.business_postal_code.isna()] # 954 rows - some are food trucks
# test_x.loc[test_x.business_postal_code.isna()] # 232 rows - some are food trucks
# train_x.loc[train_x.business_postal_code == "00000"]  # An The Go - Food Truck
# x = train_x.loc[(train_x.business_postal_code.isna() == True) & (train_x.business_address == "Off The Grid")]
# x.shape # (155, 10)
# x = train_x.loc[(train_x.business_postal_code.isna() == True) & (train_x.inspection_type == "Routine - Unscheduled")]
# x.shape # (668, 10)

# give all null values a 00000 zipcode - most appear to be food trucks without a fixed address
train_x.loc[train_x.business_postal_code.isna() == True, "business_postal_code"] = "00000"
test_x.loc[test_x.business_postal_code.isna() == True, "business_postal_code"] = "00000"

########### we have some long zipcodes like 941102019 or 94122-1909

zipcode = train_x['business_postal_code'].astype(str).str[:5]
train_x['business_postal_code'] = zipcode

zipcode_t = test_x['business_postal_code'].astype(str).str[:5]
test_x['business_postal_code'] = zipcode_t

# test_x has a zipcode "0" for the An the Go foodtruck
test_x.loc[test_x.business_postal_code == "0", "business_postal_code"] = "00000"

train_x.business_postal_code.unique()

array(['94133', '94118', '94108', '94112', '94102', '94114', '94103',
       '94107', '94110', '94104', '00000', '94105', '94115', '94123',
       '94134', '94122', '94117', '94121', '94130', '94124', '94116',
       '94158', '94111', '94109', '94127', '94131', '95133', '94132',
       '94188', '95122', '95109', '94101', '95117', '94901', '94602',
       '94013', '94080', '95132', '94143', '94544', '64110', '94301',
       '94518', '94621', '94129', '95105', '94120', '95112', '94014',
       '92672'], dtype=object)

In [10]:
# check remaining null values
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43199 entries, 0 to 43198
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   business_id            43199 non-null  int64 
 1   business_name          43199 non-null  object
 2   business_address       43199 non-null  object
 3   business_postal_code   43199 non-null  object
 4   inspection_id          43199 non-null  object
 5   inspection_date        43199 non-null  object
 6   inspection_type        43199 non-null  object
 7   violation_id           32218 non-null  object
 8   violation_description  32218 non-null  object
 9   risk_category          32218 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.3+ MB


In [11]:
# remove rows that are restaurant / inspection specific

# train_x.violation_id.nunique() -> 32183                   # almost equal to the 32218 non-null values, remove
# train_x.violation_description.nunique() -> 65             # label encode
# train_x.risk_category.nunique() -> 3                      # label encode

train_x = train_x.drop(["violation_id", "inspection_id", "inspection_date", "business_id", "business_name", "business_address"], axis = 1)
test_x = test_x.drop(["violation_id", "inspection_id", "inspection_date", "business_id", "business_name", "business_address"], axis = 1)

train_x.shape, test_x.shape

((43199, 4), (10774, 4))

In [12]:
# deal with missing values: violation_description, risk_category

# put in the string "None" for null values in violation description and risk category for restaurants 
# that didn't get any inspection score or had no violations

train_x.loc[(train_x.violation_description.isna() == True), "violation_description"] = "None" 
test_x.loc[(test_x.violation_description.isna() == True), "violation_description"] = "None" 

train_x.loc[(train_x.risk_category.isna() == True), "risk_category"] = "None" 
test_x.loc[(test_x.risk_category.isna() == True), "risk_category"] = "None" 
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43199 entries, 0 to 43198
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   business_postal_code   43199 non-null  object
 1   inspection_type        43199 non-null  object
 2   violation_description  43199 non-null  object
 3   risk_category          43199 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [13]:
# encode risk_category, violation_description and inspection_type 
# // used apply instead of LabelEncode because there were categories in test_x that were not in train_x

# risk category
risk_cat = train_x.risk_category.unique().tolist()
train_x["risk_category"] = train_x['risk_category'].apply(risk_cat.index)
test_x["risk_category"] = test_x['risk_category'].apply(risk_cat.index)

# violation description
violation_desc = train_x.violation_description.unique().tolist()
train_x["violation_description"] = train_x["violation_description"].apply(violation_desc.index)
test_x["violation_description"] = test_x["violation_description"].apply(violation_desc.index)

# inspection type
inspection_t = train_x.inspection_type.unique().tolist()
inspection_t.append("Community Health Assessment")

train_x["inspection_type"] = train_x["inspection_type"].apply(inspection_t.index)
test_x["inspection_type"] = test_x["inspection_type"].apply(inspection_t.index)

train_x

,business_postal_code,inspection_type,violation_description,risk_category
0,94133,0,0,0
1,94118,1,1,0
2,94108,1,2,0
3,94112,2,3,1
4,94102,1,4,0
...,...,...,...,...
43194,94107,1,1,0
43195,94132,3,3,1
43196,94105,1,23,0
43197,94112,1,27,0


In [14]:
# Since we have the inspection score of our training data, we are going to be using a supervised learning algorithm

# using regression tree with bagging
from sklearn.ensemble import BaggingRegressor

bagging_regressor = BaggingRegressor(n_estimators = 10, max_features = 4)
bagging_regressor.fit(train_x, train_y)

print("Bagging Regressor score is: ", bagging_regressor.score(train_x, train_y))

Bagging Regressor score is:  0.966433275495685


In [15]:
# get root mean squared error from my training data and the models prediction

pred_y_train = bagging_regressor.predict(train_x)

from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(train_y, pred_y_train))
rmse

7.196439627186866

In [16]:
# get test_y values
test_y = bagging_regressor.predict(test_x)
test_y = pd.Series(test_y)

# change -1 values to null
for i in range(test_y.shape[0]):
    if test_y[i] == -1:
        test_y[i] = None

test_y = pd.DataFrame(test_y, columns = ["inspection_score"])
test_y

,inspection_score
0,NaN
1,NaN
2,NaN
3,NaN
4,86.167183
...,...
10769,81.277399
10770,82.514679
10771,NaN
10772,NaN


In [17]:
# summary of bagging regression tree model 
bagging_regressor.get_params

<bound method BaseEstimator.get_params of BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=4, max_samples=1.0, n_estimators=10, n_jobs=None,
                 oob_score=False, random_state=None, verbose=0,
                 warm_start=False)>

In [18]:
# save predicted values to csv
test_y.iloc[:, 0].to_csv("preds.csv", index = False)